# Database overview notebook

> First time use: follow instructions in the README.md file in this directory.

## Setup

In [284]:
from timelink.mhk.utilities import get_engine, get_dbnames, get_connection_string
from sqlalchemy import engine, text, MetaData
print('Available databases:')
print(get_dbnames())
#%load_ext sql
#%config SqlMagic.displaycon=False

Available databases:
['alunos', 'angelica', 'api_tests', 'china', 'china_coimbra', 'dyncoopnet', 'eiras', 'familias', 'glopes', 'ilhavo', 'ilhavo2', 'ilhavo_contrib', 'ilhavo_editor', 'ilhavo_server', 'ilhavo_testes', 'lousa', 'lousa3', 'lousa4', 'mhk', 'notaveis', 'obidos', 'oguest', 'rhv', 'santaclara', 'santajusta', 'sister', 'soure', 'soure_edit', 'soure_editor', 'soure_mfg', 'soure_public', 'soure_server', 'testes', 'toliveira', 'toliveira_reviewed', 'ucprosop']


## Connect to database

### Choose the database

In [285]:
db = 'soure_editor'
connection_string = get_connection_string(db)
engine = get_engine(db)

## Database status

In [286]:
with engine.connect() as connection:
    classes = connection.execute(
        text('select class,count(*) as n from entities group by class')
    )
    classes
for c in classes:
    print(c['class'],c.n)

act 10842
acta 180
acusacoes 375
aforamento 2
aregister 2
attribute 66539
cartaperdao 2
caso 49
class 27
crime 1
divida 3
escritura 253
geoentity 9
good 6
group_element 10869
item 92
object 2
person 52228
relation 104592
rperson 1124
source 193


## Map MHK database

In [287]:

from sqlalchemy import Column,String,Integer,DateTime, ForeignKey
from sqlalchemy.orm import declarative_base, relationship

Base = declarative_base()


## Entities
     
        |-----------|--------------|------|-----|---------------------+-----------------------------+
        | Field     | Type         | Null | Key | Default             | Extra                       |
        +-----------+--------------+------+-----+---------------------+-----------------------------+
        | id        | varchar(64)  | NO   | PRI | NULL                |                             |
        | class     | varchar(64)  | YES  | MUL | NULL                |                             |
        | inside    | varchar(64)  | YES  | MUL | NULL                |                             |
        | the_order | decimal(6,0) | YES  |     | NULL                |                             |
        | the_level | decimal(6,0) | YES  |     | NULL                |                             |
        | the_line  | decimal(6,0) | YES  |     | NULL                |                             |
        | groupname | varchar(32)  | YES  | MUL | NULL                |                             |
        | updated   | timestamp    | NO   | MUL | CURRENT_TIMESTAMP   | on update CURRENT_TIMESTAMP |
        | indexed   | timestamp    | NO   | MUL | 1974-04-25 00:00:01 |                             |
        +-----------+--------------+------+-----+---------------------+-----------------------------+

In [288]:

class Entity(Base):
    __tablename__ = 'entities'

    id = Column(String, primary_key=True)
    pom_class = Column('class',String)
    inside = Column(String)
    the_order = Column(Integer)
    the_level = Column(Integer)
    the_line = Column(Integer)
    groupname = Column(String)
    updated = Column(DateTime)
    indexed = Column(DateTime)


    rels_in = relationship("Relation", back_populates="dest")
    rels_out = relationship("Relation",back_populates="org")

    # see https://docs.sqlalchemy.org/en/14/orm/inheritance.html
    __mapper_args__ = {
        'polymorphic_identity': 'entity',
        'polymorphic_on':pom_class
    }

    # untested
    @classmethod  # untested
    def get_subclasses(cls):
        for subclass in cls.__subclasses__():
            yield from subclass.get_subclasses()
            yield subclass

    def __repr__(self):
        return (
            f'Entity(id="{self.id}", '
            f'pom_class="{self.pom_class}",'
            f'inside="{self.inside}", '
            f'the_order={self.the_order}, '
            f'the_level={self.the_level}, '
            f'the_line={self.the_line}, '
            f'groupname="{self.groupname}", '
            
            f'updated={self.updated}, '
            f'indexed={self.indexed},'
            f')'
        )
        
    def __str__(self):
        return (f'{self.groupname}${self.id}/type={self.pom_class}')


## Attributes
    +-----------+----------------+------+-----+---------+-------+
    | Field     | Type           | Null | Key | Default | Extra |
    +-----------+----------------+------+-----+---------+-------+
    | id        | varchar(64)    | NO   | PRI | NULL    |       |
    | entity    | varchar(64)    | YES  | MUL | NULL    |       |
    | the_type  | varchar(512)   | YES  | MUL | NULL    |       |
    | the_value | varchar(1024)  | YES  | MUL | NULL    |       |
    | the_date  | varchar(24)    | YES  | MUL | NULL    |       |
    | obs       | varchar(16000) | YES  |     | NULL    |       |
    +-----------+----------------+------+-----+---------+-------+

In [289]:

class Attribute(Entity):
    __tablename__ = 'attributes'

    id = Column(String, ForeignKey('entities.id'), primary_key=True)
    entity = Column(String,ForeignKey('entities.id'))
    the_type = Column(String)
    the_value = Column(String)
    the_date = Column(String)
    obs = Column(String)

    the_entity = relationship("Entity",foreign_keys=[entity],back_populates="attributes",)

    __mapper_args__ = {
        'polymorphic_identity':'attribute',     
        'inherit_condition': id == Entity.id
    }

    def __repr__(self):
        return (
            f'Attribute(id={super().__repr__()}, '
            f'entity="{self.entity}", '
            f'the_type="{self.the_type}", '
            f'the_value="{self.the_value}", '
            f'the_date="{self.the_date}"", '
            f'obs={self.obs}'
            f')'
        )

    def __str__(self):
        r = f'     ls${self.the_type}/{self.the_value}/{self.the_date}'
        if self.obs is not None:
                r = (f'{r}/obs={self.obs}')
        return r


Entity.attributes = relationship("Attribute", foreign_keys=[Attribute.entity], back_populates="the_entity")


## Relations   
    +-------------+----------------+------+-----+---------+-------+
    | Field       | Type           | Null | Key | Default | Extra |
    +-------------+----------------+------+-----+---------+-------+
    | id          | varchar(64)    | NO   | PRI | NULL    |       |
    | origin      | varchar(64)    | YES  | MUL | NULL    |       |
    | destination | varchar(64)    | YES  | MUL | NULL    |       |
    | the_date    | varchar(24)    | YES  | MUL | NULL    |       |
    | the_type    | varchar(32)    | YES  | MUL | NULL    |       |
    | the_value   | varchar(256)   | YES  | MUL | NULL    |       |
    | obs         | varchar(16000) | YES  |     | NULL    |       |
    +-------------+----------------+------+-----+---------+-------+

In [290]:
class Relation(Entity):  # should extend Entity but gives error

    __tablename__ = 'relations'

    id = Column(String,ForeignKey('entities.id'), primary_key=True)
    #rel_entity = relationship("Entity",foreign_keys='id',back_populates='rel')
    origin = Column(String,ForeignKey('entities.id'))
    org = relationship(Entity,foreign_keys=[origin], back_populates='rels_out')

    destination = Column(String,ForeignKey('entities.id'))
    dest = relationship("Entity",foreign_keys=[destination], back_populates="rels_in")
    the_type = Column(String)
    the_value = Column(String)
    the_date = Column(String)
    obs = Column(String)

    __mapper_args__ = {
        'polymorphic_identity':'relation',
        'inherit_condition': id == Entity.id
    }

    def __repr__(self):
        return (
            f'Relation(id={super().__repr__()}, '
            f'origin="{self.origin}", '
            f'destination="{self.destination}", '
            f'the_type="{self.the_type}", '
            f'the_value="{self.the_value}", '
            f'the_date="{self.the_date}"", '
            f'obs={self.obs}'
            f')'
        )

    def __str__(self):
        if self.dest is not None and self.dest.pom_class == 'person':
            r = f'rel${self.the_type}/{self.the_value}/{self.dest.name}/{self.destination}/{self.the_date}'
        else:
            r = f'rel${self.the_type}/{self.the_value}/{self.destination}/{self.the_date}'
        if self.obs is not None:
                r = (f'{r}  /obs={self.obs}')
        return r

Entity.rels_out = relationship("Relation",foreign_keys=[Relation.origin],back_populates="dest")
Entity.rels_in = relationship("Relation",foreign_keys=[Relation.destination],back_populates="org")

/var/folders/sx/x6pyh9097n98mlz29_6njdj00000gn/T/ipykernel_3272/4209139393.py:43: SAWarning: Property Entity.rels_out on mapped class Entity->entities being replaced with new property Entity.rels_out; the old property will be discarded
  Entity.rels_out = relationship("Relation",foreign_keys=[Relation.origin],back_populates="dest")
/var/folders/sx/x6pyh9097n98mlz29_6njdj00000gn/T/ipykernel_3272/4209139393.py:43: SAWarning: Property Entity.rels_out on mapped class Attribute->attributes being replaced with new property Entity.rels_out; the old property will be discarded
  Entity.rels_out = relationship("Relation",foreign_keys=[Relation.origin],back_populates="dest")
/var/folders/sx/x6pyh9097n98mlz29_6njdj00000gn/T/ipykernel_3272/4209139393.py:43: SAWarning: Property Entity.rels_out on mapped class Relation->relations being replaced with new property Entity.rels_out; the old property will be discarded
  Entity.rels_out = relationship("Relation",foreign_keys=[Relation.origin],back_populate

## Persons    
    +-------+----------------+------+-----+---------+-------+
    | Field | Type           | Null | Key | Default | Extra |
    +-------+----------------+------+-----+---------+-------+
    | id    | varchar(64)    | NO   | PRI | NULL    |       |
    | name  | varchar(128)   | YES  | MUL | NULL    |       |
    | sex   | char(1)        | YES  |     | NULL    |       |
    | obs   | varchar(16000) | YES  |     | NULL    |       |
    +-------+----------------+------+-----+---------+-------+

In [291]:
class Person(Entity):
    __tablename__ = 'persons'

    id = Column(String, ForeignKey('entities.id'), primary_key=True)
    name = Column(String)
    sex = Column(String(1))
    obs = Column(String)

    __mapper_args__ = {
        'polymorphic_identity':'person'
    }


    def __str__(self):
        r = f'{self.groupname}${self.name}/{self.sex}/id={self.id}'
        if self.obs is not None:
                r = (f'{r}  /obs={self.obs}')
        return r

## Objects

    +----------+----------------+------+-----+---------+-------+
    | Field    | Type           | Null | Key | Default | Extra |
    +----------+----------------+------+-----+---------+-------+
    | id       | varchar(64)    | NO   | PRI | NULL    |       |
    | name     | varchar(64)    | YES  | MUL | NULL    |       |
    | the_type | varchar(32)    | YES  |     | NULL    |       |
    | obs      | varchar(16000) | YES  |     | NULL    |       |
    +----------+----------------+------+-----+---------+-------+

In [292]:
class Object(Entity):
    __tablename__ = 'objects'

    id = Column(String, ForeignKey('entities.id'), primary_key=True)
    name = Column(String)
    the_type = Column(String(32))
    obs = Column(String)

    __mapper_args__ = {
        'polymorphic_identity':'object'
    }

## Acts

    +----------+----------------+------+-----+---------+-------+
    | Field    | Type           | Null | Key | Default | Extra |
    +----------+----------------+------+-----+---------+-------+
    | id       | varchar(64)    | NO   | PRI | NULL    |       |
    | the_type | varchar(32)    | YES  |     | NULL    |       |
    | the_date | varchar(24)    | YES  | MUL | NULL    |       |
    | loc      | varchar(64)    | YES  |     | NULL    |       |
    | ref      | varchar(64)    | YES  |     | NULL    |       |
    | obs      | varchar(16000) | YES  |     | NULL    |       |
    +----------+----------------+------+-----+---------+-------+


In [293]:
class Act(Entity):
    __tablename__ = 'acts'

    id = Column(String, ForeignKey('entities.id'), primary_key=True)
    the_type = Column(String(32))
    the_date = Column(String)
    loc = Column(String)
    ref = Column(String)
    obs = Column(String)

    __mapper_args__ = {
        'polymorphic_identity':'act'
    }

    def __str__(self):
        r = f'{self.groupname}${self.id}/{self.the_date}/type={self.the_type}/ref={self.ref}/loc={self.loc}'
        if self.obs is not None:
                r = (f'{r}  /obs={self.obs}')
        return r


## Sources

    +-----------+----------------+------+-----+---------+-------+
    | Field     | Type           | Null | Key | Default | Extra |
    +-----------+----------------+------+-----+---------+-------+
    | id        | varchar(64)    | NO   | PRI | NULL    |       |
    | the_type  | varchar(32)    | YES  |     | NULL    |       |
    | the_date  | varchar(24)    | YES  |     | NULL    |       |
    | loc       | varchar(64)    | YES  |     | NULL    |       |
    | ref       | varchar(64)    | YES  |     | NULL    |       |
    | kleiofile | varchar(1024)  | YES  |     | NULL    |       |
    | replaces  | varchar(254)   | YES  |     | NULL    |       |
    | obs       | varchar(16000) | YES  |     | NULL    |       |
    +-----------+----------------+------+-----+---------+-------+

In [294]:
class Source(Entity):
    __tablename__ = 'sources'

    id = Column(String, ForeignKey('entities.id'), primary_key=True)
    the_type = Column(String(32))
    the_date = Column(String)
    loc = Column(String)
    ref = Column(String)
    kleiofile = Column(String)
    replaces = Column(String)
    obs = Column(String)

    __mapper_args__ = {
        'polymorphic_identity':'source'
    }

    def __str__(self):
        r = f'{self.groupname}${self.id}/{self.the_date}/type={self.the_type}/ref={self.ref}/loc={self.loc}/kleiofile={self.kleiofile}/replaces={self.replaces}'
        if self.obs is not None:
                r = (f'{r}  /obs={self.obs}')
        return r

## Classes

    +------------+-------------+------+-----+---------+-------+
    | Field      | Type        | Null | Key | Default | Extra |
    +------------+-------------+------+-----+---------+-------+
    | id         | varchar(64) | NO   | PRI | NULL    |       |
    | table_name | varchar(32) | YES  |     | NULL    |       |
    | group_name | varchar(32) | YES  |     | NULL    |       |
    | super      | varchar(64) | YES  |     | NULL    |       |
    +------------+-------------+------+-----+---------+-------+



In [295]:
class PomClass(Entity):
    __tablename__ = 'classes'

    id = Column(String, ForeignKey('entities.id'), primary_key=True)
    table_name = Column(String)
    class_group = Column("group_name",String(32))
    super_class = Column("super", String)

    __mapper_args__ = {
        'polymorphic_identity':'class'
    }

    def __repr__(self):
        return (
            f'PomClass(id={self.id}, '
            f'table_name="{self.table_name}", '
            f'class_group="{self.class_group}", '
            f'super_class="{self.super_class}" '
            f')'
        )

    def __str__(self):
        r = f'{self.id} table {self.table_name}\n'
        for cattr in self.class_attributes:
            r = r + f'{cattr.the_class}.{cattr.name} \tclass {cattr.colclass} \tcol {cattr.colname} \ttype {cattr.coltype} size {cattr.colsize} precision {cattr.colprecision} primary key {cattr.pkey} \n'
        return r


## Class attributes

    +--------------+--------------+------+-----+---------+-------+
    | Field        | Type         | Null | Key | Default | Extra |
    +--------------+--------------+------+-----+---------+-------+
    | the_class    | varchar(64)  | YES  | MUL | NULL    |       |
    | name         | varchar(32)  | YES  |     | NULL    |       |
    | colname      | varchar(32)  | YES  |     | NULL    |       |
    | colclass     | varchar(32)  | YES  |     | NULL    |       |
    | coltype      | varchar(32)  | YES  |     | NULL    |       |
    | colsize      | decimal(5,0) | YES  |     | NULL    |       |
    | colprecision | decimal(6,0) | YES  |     | NULL    |       |
    | pkey         | decimal(1,0) | YES  |     | NULL    |       |
    +--------------+--------------+------+-----+---------+-------+


In [296]:
class PomClassAttributes(Base):
    __tablename__ = 'class_attributes'

    the_class = Column(String, ForeignKey('classes.id'),primary_key=True)

    pom_class = relationship("PomClass",foreign_keys=[the_class], back_populates='class_attributes')
    name = Column(String(32), primary_key=True)
    colname = Column(String(32))
    colclass = Column(String(32))
    coltype = Column(String)
    colsize = Column(Integer)
    colprecision = Column(Integer)
    pkey = Column(Integer)

PomClass.class_attributes = relationship("PomClassAttributes", back_populates="pom_class")

## Testing

In [297]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()


In [298]:
attribute = session.query(Attribute).where(Entity.pom_class  == 'attribute').first()
print(attribute)
print(repr(attribute))

     ls$residencia/barrosas/16970205/obs=%barrosos. 
Attribute(id=Entity(id="$c51-2-att3-12", pom_class="attribute",inside="$c51-2", the_order=164, the_level=5, the_line=136, groupname="ls", updated=2021-08-21 08:33:00, indexed=1974-04-25 00:00:01,), entity="$c51-2", the_type="residencia", the_value="barrosas", the_date="16970205"", obs=%barrosos. )


In [299]:
entity = attribute.the_entity
print(entity)

pnoivo$domingos andre/m/id=$c51-2


In [300]:
relation = session.query(Relation).where(Relation.pom_class  == 'relation').first()

print(repr(relation))
print(relation)


Relation(id=Entity(id="$c51-2-att2-2-geo-rel", pom_class="relation",inside="$c51-2-att2-2-geo", the_order=None, the_level=None, the_line=None, groupname="*georelation", updated=2020-02-26 21:55:45, indexed=1974-04-25 00:00:01,), origin="$c51-2-att2-2-geo", destination="$c51-2", the_type="geografica", the_value="residencia", the_date="16970205"", obs=*automatic relation)
rel$geografica/residencia/domingos andre/$c51-2/16970205  /obs=*automatic relation


In [301]:
relation = session.query(Relation).where(Relation.the_type  == 'parentesco').first()
print(repr(relation))

print(relation.org)
print("   ",relation)
print(relation.dest)

Relation(id=Entity(id="01705-a5e-rel17-4", pom_class="relation",inside="01705-a5e", the_order=274, the_level=5, the_line=331, groupname="rel", updated=2021-08-21 08:46:22, indexed=1974-04-25 00:00:01,), origin="01705-a5e", destination="1705-a5ea", the_type="parentesco", the_value="foi mulher", the_date="17051005"", obs=None)
referida$maria fernandes/f/id=01705-a5e
    rel$parentesco/foi mulher/manuel cordeiro/1705-a5ea/17051005
referido$manuel cordeiro/m/id=1705-a5ea


In [302]:
act = session.query(Act).where(Entity.pom_class  == 'act').first()

print(repr(act))
print(act)

Entity(id="1700-142", pom_class="act",inside="casamentos 1700", the_order=2, the_level=2, the_line=6, groupname="cas", updated=2020-06-17 18:18:21, indexed=2020-06-17 18:18:21,)
cas$1700-142/17000110/type=cas/ref=igreja matriz/loc=luis alvares pinto


In [303]:
source = session.query(Source).where(Entity.pom_class  == 'source').first()

print(repr(source))
print(source)

Entity(id="1D-8-2-4", pom_class="source",inside="root", the_order=1, the_level=1, the_line=4, groupname="fonte", updated=2021-11-02 11:06:17, indexed=1974-04-25 00:00:01,)
fonte$1D-8-2-4/16910314/type=livro de notas/ref=None/loc=AUC D-8-2-4/kleiofile=/kleio-home/sources/soure-fontes/sources/1685-1720/notariais/1D-8-2-4.cli/replaces=n2.23  /obs=legivel


In [304]:
pom_classes = session.query(PomClass).where(Entity.pom_class  == 'class').all()
for pom_class in pom_classes:
    print(repr(pom_class))
    print(pom_class)

PomClass(id=act, table_name="acts", class_group="historical-act", super_class="entity" )
act table acts
act.date class date 	col the_date 	type varchar size 24 precision 0 primary key 0 
act.id class id 	col id 	type varchar size 64 precision 0 primary key 1 
act.loc class loc 	col loc 	type varchar size 64 precision 0 primary key 0 
act.obs class obs 	col obs 	type varchar size 16654 precision 0 primary key 0 
act.ref class ref 	col ref 	type varchar size 64 precision 0 primary key 0 
act.type class type 	col the_type 	type varchar size 32 precision 0 primary key 0 

PomClass(id=acta, table_name="actas", class_group="amz", super_class="act" )
acta table actas
acta.ano class year 	col the_year 	type numeric size 4 precision 0 primary key 0 
acta.dia class day 	col the_day 	type numeric size 2 precision 0 primary key 0 
acta.fol class fol 	col fol 	type varchar size 64 precision 0 primary key 0 
acta.id class id 	col id 	type varchar size 64 precision 0 primary key 1 
acta.mes class mon

In [65]:
print(Entity.__subclasses__())
for aclass in Entity.__subclasses__():
    print(aclass.__tablename__)

[<class '__main__.Person'>]
persons


In [63]:
from sqlalchemy import select
from sqlalchemy.orm import Session

stmt = select(Person).first()
print(stmt)
with Session(engine) as session:
    for row in session.execute(stmt):
        entity: Entity=row[0]
        print(f'{entity.groupname}${entity.name}/id={entity.id}#{entity.pom_class}')
        for attribute in entity.attributes:
            print(f'     ls${attribute.the_type}/{attribute.the_value}/{attribute.the_date}',end='')
            if attribute.obs is not None:
                print(f'/obs={attribute.obs}')
            else:
                print()
        if len(entity.rels_out)>0:
            for rel in entity.rels_out:
                print(f'>{rel}')
        if len(entity.rels_in)>0:
            for rel in entity.rels_in:
                    print(rel,'<')               


SELECT entities.class, persons.id, entities.id AS id_1, entities.inside, entities.the_order, entities.the_level, entities.the_line, entities.groupname, entities.updated, entities.indexed, persons.name, persons.sex, persons.obs 
FROM entities JOIN persons ON entities.id = persons.id 
WHERE persons.id = :id_2
n$Arnaldo Henriques de Abreu/id=140625#person
     ls$código-de-referência/"PT/AUC/ELU/UC-AUC/B/001-001/A/000336"/20200211
     ls$data-do-registo/2018-04-19 14:33:58/20200211
     ls$url/"https://pesquisa.auc.uc.pt/details?id=140625"/20200211
     ls$uc.entrada/1835-10-17/1835-10-17
     ls$uc.saida/1838-10-09/1838-10-09
     ls$uc.entrada.ano/1835/1835-10-17
     ls$uc.saida.ano/1838/1838-10-09
     ls$nome/Arnaldo Henriques de Abreu/1835-10-17
     ls$nome.primeiro/Arnaldo/1835-10-17
     ls$nome.apelido/Henriques de Abreu/1835-10-17
     ls$nome.apelido/Abreu/1835-10-17
     ls$naturalidade/Porto/1835-10-17
     ls$naturalidade.ano/Porto.1835/1835-10-17
     ls$nome-geografico/P

## Source files

In [37]:
from pathlib import Path

kleio_files = [f.stem for f in list(Path('../sources').rglob('*.cli'))]
print("Number of kleio_files:", len(kleio_files))

imported_sources = %sql select sources.id, sources.kleiofile, entities.updated from sources left join entities on (sources.id=entities.id) order by updated desc 
sources_in_db = [s.id for s in imported_sources]
print("Number of imported files:",len(sources_in_db))

print("Files not in the database:", set(kleio_files)-set(sources_in_db))
print("Imported sources no file found:", set(sources_in_db)-set(kleio_files))



Number of kleio_files: 235
235 rows affected.
Number of imported files: 235
Files not in the database: set()
Imported sources no file found: set()


## Analyse attributes extracted from records

### Attributes in the database

In [8]:
nml = %sql select the_type, count(*) as tot from attributes group by the_type
for the_type, tot in nml:
    print(f'{tot:6} | {the_type}')

28 rows affected.
105300 | código-de-referência
   275 | colegio
105300 | data-do-registo
   186 | ec
 53627 | exame
 94164 | faculdade
 94164 | faculdade.ano
106372 | grau
106372 | grau.ano
319291 | matricula
319291 | matricula.ano
 98904 | naturalidade
 98904 | naturalidade.ano
105300 | nome
107486 | nome-geografico
101392 | nome-geografico.ano
198660 | nome.apelido
105300 | nome.primeiro
  8155 | nome.vide
  9975 | nota
  3142 | ordem-religiosa
  7036 | padre
  4170 | titulo
105300 | uc.entrada
 99014 | uc.entrada.ano
105300 | uc.saida
 99281 | uc.saida.ano
105300 | url


In [11]:
p1540 = %sql select id, name, pobs from nattributes where the_type='uc.entrada' and the_value like '1540%'
for id, name, pobs in p1540:
    print(f'{id:5} | {name} \n {pobs}\n\n')

473 rows affected.


ValueError: Unknown format code 'r' for object of type 'str'